Question 1
Running Elastic Search 8.17.6
Running this in Visual Studio Code
docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.17.6

Question 2

In [101]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [102]:
import elasticsearch
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://localhost:9200")
info = es_client.info()
print("Elasticsearch version:", info['version']['number'])

Elasticsearch version: 8.17.6


In [103]:
index_settings ={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
if es_client.indices.exists(index="course-questions"):
    es_client.indices.delete(index="course-questions")
index_name = "course-questions"
es_client.indices.create(
    index=index_name,
    body=index_settings
)

/tmp/ipykernel_6213/2215345132.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [104]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 344.75it/s]


Question 3

In [135]:
def elastic_search(query, course):
    #Search query
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
               
            }
        }
    }
    
    #response from elastic search
    response = es_client.search(
        index=index_name,
        body=search_query
    )
    
    #Let's print the response's top ranking result
    top_score = response['hits']['hits'][0]['_score']
    return top_score

In [136]:
#Running this query 'How do execute a command on a Kubernetes pod?'
query = 'How do execute a command on a Kubernetes pod?'
course = "machine-learning-zoomcamp"
score = elastic_search(query, course)
print('For query', query, 'the top score is:', score)

For query How do execute a command on a Kubernetes pod? the top score is: 44.50556


/tmp/ipykernel_6213/2227997108.py:20: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(


Question 4

In [107]:
#Running this query 'How do I copy a file to a Docker container?' with machine-learning-zoomcamp as the filter
query = 'How do copy a file to a Docker container?'
course = 'machine-learning-zoomcamp'
score = elastic_search(query,course)
print('For query', query, 'the top score is:', score)

For query How do copy a file to a Docker container? the top score is: 73.38676


/tmp/ipykernel_6213/4215983962.py:24: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(


In [108]:
def elastic_search(query):
    #Search query
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": 'machine-learning-zoomcamp'
                    }
                }
            }
        }
    }
    
    #response from elastic search
    response = es_client.search(
        index=index_name,
        body=search_query
    )
    
    #Let's print the response's third question
    hits = response['hits']['hits']
    third_question = hits[2]['_source']['question']
    return third_question

In [122]:
query = 'How do copy a file to a Docker container?'
result = elastic_search(query)
print('The 3rd question returned by the search engine for', query, 'is:', result)

The 3rd question returned by the search engine for How do copy a file to a Docker container? is: How do I copy files from a different folder into docker container’s working directory?


/tmp/ipykernel_6213/1759229725.py:24: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(


Question 5

In [123]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [124]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [125]:
from openai import OpenAI
import os
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [126]:
def prompt(query):
    search_query = {
            "size": 3,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {
                        "term": {
                            "course": 'machine-learning-zoomcamp'
                        }
                    }
                }
            }
        }
        
    #response from elastic search
    response = es_client.search(
        index=index_name,
        body=search_query
    )
    #Given the prompt and context template we are constructing a prompt
    context = ""
    for doc in response['hits']['hits']:
        source = doc['_source']
        context += context_template.format(
            question=source.get('question', ''),
            text=source.get('text', '')
        ) + "\n\n"  # two line breaks between entries

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt, len(prompt)

In [127]:
query = 'How do copy a file to a Docker container?'
prompt_generated, length = prompt(query)
print('This is the prompt generated')
print(prompt_generated)
print('Length is:', length)

This is the prompt generated
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory 

/tmp/ipykernel_6213/1461047944.py:23: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(


Question 6

In [128]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [129]:
tokens = encoding.encode(prompt_generated)
# Print number of tokens
print("Number of tokens in prompt:", len(tokens))

Number of tokens in prompt: 320


In [130]:
#Let us send the prompt to OpenAI
def run_llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user', 'content':prompt}]
    )
    answer = response.choices[0].message.content
    return answer
answer = run_llm(prompt_generated)
print(query)
print('OpenAI gpt-4o response: ', answer)

How do copy a file to a Docker container?
OpenAI gpt-4o response:  To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax to copy a file or directory from your local machine into a running Docker container is as follows:

```
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```

This command allows you to transfer files from your local machine into a specific location inside the container.
